# **Dataset Preprocessing**

## **Import Libraries**

In [9]:
import json
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords

In [10]:
# jika belum pernah:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## **Load Dataset**

In [ ]:
input_file = "cqa_datasets.jsonl"
df = pd.read_json(input_file, lines=True)
print(f"Total QA pairs: {len(df)}")
df.head(3)

Total QA pairs: 868


,context,question,answer,file_url,regulation_number,title,filename,n_pairs_requested
0,Peraturan Otoritas Jasa Keuangan Nomor 27 Tahu...,Apa tujuan dari Peraturan Otoritas Jasa Keuang...,Tujuan dari peraturan ini adalah untuk mencipt...,https://www.ojk.go.id/id/regulasi/Documents/Pa...,27 Tahun 2022,Perubahan Kedua Atas Peraturan Otoritas Jasa K...,ojk-peraturan_ojk-27_tahun_2022-28122022-perub...,3
1,Perubahan yang diatur dalam Peraturan ini menc...,Apa saja perubahan yang diatur dalam Peraturan...,Perubahan tersebut mencakup kewajiban bank unt...,https://www.ojk.go.id/id/regulasi/Documents/Pa...,27 Tahun 2022,Perubahan Kedua Atas Peraturan Otoritas Jasa K...,ojk-peraturan_ojk-27_tahun_2022-28122022-perub...,3
2,Peraturan ini juga mengatur tentang penyediaan...,Apa yang harus dilakukan bank mulai 1 Januari ...,"Mulai 1 Januari 2024, seluruh bank wajib mempe...",https://www.ojk.go.id/id/regulasi/Documents/Pa...,27 Tahun 2022,Perubahan Kedua Atas Peraturan Otoritas Jasa K...,ojk-peraturan_ojk-27_tahun_2022-28122022-perub...,3


## **Filter Data with High Similarity Context**

In [22]:
# 1) Ambil stop-words Bahasa Indonesia
stop_id = set(stopwords.words("indonesian"))

# 2) Hitung TF-IDF
vectorizer = TfidfVectorizer(
    max_features=5000,
    stop_words=list(stop_id),
    token_pattern=r"(?u)\b\w+\b"
)
tfidf_matrix = vectorizer.fit_transform(df['context'])

# 3) Cari semua duplikat di atas threshold
threshold = 0.8
pairs_over_threshold = []  # list of (idx1, idx2, sim)

for start in tqdm(range(0, tfidf_matrix.shape[0], 500), desc="Scanning blocks"):
    end   = min(start+500, tfidf_matrix.shape[0])
    block = tfidf_matrix[start:end]
    sims  = cosine_similarity(block, tfidf_matrix)
    for i, sims_row in enumerate(sims):
        idx1 = start + i
        high = np.where(sims_row > threshold)[0]
        for idx2 in high:
            if idx2 > idx1:
                pairs_over_threshold.append((idx1, idx2, sims_row[idx2]))

print(f"Ketemu {len(pairs_over_threshold)} pasangan duplikat >{threshold}\n")

# 4) Tampilkan beberapa contoh untuk inspeksi (konteks penuh)
for a, b, sim in pairs_over_threshold[:10]:
    print(f"- idx {a} ↔ idx {b}: sim={sim:.3f}\n")
    print(f"  * Context A (idx={a}):\n{df.loc[a, 'context']}\n")
    print(f"  * Context B (idx={b}):\n{df.loc[b, 'context']}\n")
    print("-" * 80)

# 5) Bangun union-find untuk cluster duplikat
parent = list(range(len(df)))
def find(x):
    while parent[x] != x:
        parent[x] = parent[parent[x]]
        x = parent[x]
    return x
def union(a, b):
    ra, rb = find(a), find(b)
    if ra != rb:
        parent[rb] = ra

for a, b, _ in pairs_over_threshold:
    union(a, b)

# 6) Kelompokkan berdasarkan root
clusters = {}
for idx in range(len(df)):
    r = find(idx)
    clusters.setdefault(r, []).append(idx)

# *** Verifikasi klaster besar ***
print("\nKlaster dengan >2 anggota:")
for root, members in clusters.items():
    if len(members) > 2:
        print(f"  root={root}: anggota={members}")

# 7) Pilih konteks terpanjang per klaster
to_keep = []
for members in clusters.values():
    if len(members) == 1:
        to_keep.append(members[0])
    else:
        longest = max(members, key=lambda i: len(df.loc[i, 'context']))
        to_keep.append(longest)

# 8) Bangun DataFrame hasil filter
df_filtered = df.loc[sorted(to_keep)].reset_index(drop=True)
print(f"\nAfter filtering clusters: {len(df_filtered)} pairs remain")

# 9) Simpan hasil
output_file = "cqa_datasets_clean.jsonl"
df_filtered.to_json(output_file, orient="records", lines=True)
print(f"Filtered data saved to {output_file}")

c:\Users\acer\miniconda3\envs\ta-datasets-creation\lib\site-packages\sklearn\feature_extraction\text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(
Scanning blocks: 100%|██████████| 2/2 [00:00<00:00, 107.92it/s]

Ketemu 9 pasangan duplikat >0.8

- idx 14 ↔ idx 544: sim=0.834

  * Context A (idx=14):
Peraturan ini juga mencakup sanksi administratif bagi pihak yang melanggar ketentuan yang ada. Sanksi ini dapat berupa peringatan tertulis, denda, pembatasan kegiatan usaha, hingga pencabutan izin usaha. Otoritas Jasa Keuangan memiliki wewenang untuk menjatuhkan sanksi kepada pihak yang melanggar, termasuk pihak yang menyebabkan pelanggaran tersebut terjadi. Hal ini bertujuan untuk memastikan kepatuhan terhadap peraturan yang ditetapkan.

  * Context B (idx=544):
Sanksi administratif juga diatur dalam peraturan ini bagi pihak yang melanggar ketentuan. Sanksi dapat berupa denda, pembatasan kegiatan usaha, hingga pencabutan izin usaha. Otoritas Jasa Keuangan memiliki wewenang untuk menjatuhkan sanksi kepada pihak yang melanggar.

--------------------------------------------------------------------------------
- idx 14 ↔ idx 683: sim=0.856

  * Context A (idx=14):
Peraturan ini juga mencakup sanksi adm

## **Split Data into 3 Parts (SFT, PPO, Test)**

- SFT: 60% of the data
- PPO: 30% of the data
- Testing: 10% of the data

In [23]:
train_val, test = train_test_split(
    df_filtered,
    test_size=0.10,
    random_state=42,
    shuffle=True
)
sft, ppo = train_test_split(
    train_val,
    test_size=0.30 / 0.90,
    random_state=42,
    shuffle=True
)

print(f"SFT:  {len(sft)} ({len(sft)/len(df_filtered):.1%})")
print(f"PPO:  {len(ppo)} ({len(ppo)/len(df_filtered):.1%})")
print(f"Test: {len(test)} ({len(test)/len(df_filtered):.1%})")

## **Save ke File JSONL**
sft.to_json("cqa_sft.jsonl", orient="records", lines=True, force_ascii=False)
ppo.to_json("cqa_ppo.jsonl", orient="records", lines=True, force_ascii=False)
test.to_json("cqa_test.jsonl", orient="records", lines=True, force_ascii=False)

SFT:  515 (60.0%)
PPO:  258 (30.0%)
Test: 86 (10.0%)
